# Colab的默认环境跟这里的默认环境有差异，需要安装额外的依赖

In [ ]:
!pip install black[jupyter] # black 的 Jupyter 扩展, 自动格式化 Python 代码。要使用 black 进行格式化，请在命令行中运行以下命令：black <filename>.ipynb
!pip install wget
!pip install transformers
!pip install bitsandbytes
!pip install jax
!pip install xformers
!pip install triton==2.0.0.dev20221202
!apt-get update
!apt-get install -y p7zip-full
!apt-get install -y iputils-ping
!apt-get install -y git-lfs

# 配置accelerate

In [ ]:
!chmod +x ./accelerate.sh
!./accelerate.sh

# 创建Temp文件夹

In [ ]:
import wget
import os, sys

sys.path.append("../")  # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxyCLI, proxyWget

cb = setProxyCLI()
proxy = cb["proxy"]
region = cb["region"]
proxyURL = cb["proxyURL"]

# 在当前运行的ipynb文件所在的目录下创建临时文件夹temp
def create_temp_folder(temp_folder):
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)


# 获取当前运行的 ipynb 文件所在的目录
cwd = os.getcwd()

# 在该目录下创建临时文件夹 temp
temp_folder = os.path.join(cwd, "temp")

# 如果temp文件夹不存在，则创建文件夹
create_temp_folder(temp_folder)

# 覆盖下载 fast-DreamBooth.ipynb 文件到 temp 文件夹中
file_name = "fast-DreamBooth.ipynb"
fastDreamBoothPath = os.path.join(temp_folder, file_name)
if os.path.exists(fastDreamBoothPath):
    !rm -rf $fastDreamBoothPath

url = "https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/fast-DreamBooth.ipynb"
out = os.path.join(temp_folder, file_name)

!$proxy && wget -O $out $url

# fastDreamBoothPath = proxyWget(
#     url=url,
#     out=out,
#     proxyURL=proxyURL)

# print(fastDreamBoothPath)

# 创建符号链接

In [ ]:
import os

def create_symlink(source_path, symlink_path, symlink_name):
    # 创建软链接
    try:
        os.symlink(source_path, f"{symlink_path}/{symlink_name}")
    except FileExistsError:
        # 如果软链接已经存在，则删除软链接
        if os.path.exists(f"{symlink_path}/{symlink_name}"):
            os.remove(f"{symlink_path}/{symlink_name}")
        # 再次尝试创建软链接
        os.symlink(source_path, f"{symlink_path}/{symlink_name}")

    # 验证软链接是否成功创建
    if os.path.islink(f"{symlink_path}/{symlink_name}"):
        print(f"{symlink_path}/{symlink_name} 是一个符号链接。")
        print(f"它指向: {os.readlink(f'{symlink_path}/{symlink_name}')}")
    else:
        print(f"{symlink_path}/{symlink_name} 不是一个符号链接。")
        
# 在output目录下创建一个tf_dir指向dreambooth打印的logs软连接  
create_symlink(
    source_path="/output/content/models/FuXingHao768/logs/", 
    symlink_path='/output', 
    symlink_name='tf_dir'
)

# 在Content目录下创建一个指向挂载模型的软连接
create_symlink(
    source_path='/input0', # openbayes 特有的目录，是外部数据集挂载到当前镜像的目录，编号从0~4
    symlink_path='/output/content', # fast-dreambooth.ipynb的 model download cell一般下载模型的路径
    symlink_name='stable-diffusion-v2-768' # 模型名称
)

# 清理Notebook中的输出，对Colab版中的路径进行替换，以适应openbayes或autodl的路径

In [ ]:
import json
import os
import re


def modify_file_name(file_path):
    # 获取文件名和文件扩展名
    file_name, file_ext = os.path.splitext(file_path)
    # 在文件名后加上 -modified
    modified_file_path = file_name + "-modified" + file_ext
    return modified_file_path


def replace_root_path(notebook: str, old_root: str, new_root: str) -> str:
    # 加载notebook
    with open(notebook, "r") as f:
        nb = json.load(f)

    # 替换旧的根路径为新的根路径

    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            # Initialize the flag variable
            import_added = False
            # 隐藏代码
            cell["metadata"] = {"collapsed": True}
            # 清除代码单元格的执行次数
            cell["execution_count"] = None
            # 清除代码单元格的输出
            cell["outputs"] = []
            for i, line in enumerate(cell["source"]):
                # 添加一个条件判断，只有当该行不是网址路径时才进行替换
                if not line.startswith(("http", "https")):
                    cell["source"][i] = line.replace(old_root, new_root)
                # 把引入google.colab的库的代码行标注为注释
                if "google.colab" in line:
                    cell["source"][i] = "# " + line
                # 把使用google drive盘挂载的代码行标注为注释
                if "drive.mount" in line:
                    cell["source"][i] = "# " + line
                # 如果找到 "!wget" 字符串，就将其替换为 "!proxy && wget"
                if "!wget" in line:
                    cell["source"][i] = line.replace("!wget", "!$proxy && wget")
                # 定义 proxyWget 函数（请确保已经定义了 proxyWget 函数）

                import textwrap

                if "wget.download" in line:
                    # 使用正则表达式查找所有使用了 proxyWget 函数的行
                    match = re.search(r"wget\.download\(.+\)", line)
                    if match:
                        # 获取 wget.download 函数调用的参数
                        params = match.group().strip("wget.download()")
                        # 计算新的一行代码的缩进
                        indent = " " * line.count(" ")
                        # 将 wget.download 函数调用替换为带有新参数的函数调用
                        new_line = (
                            f"{indent}proxyWget({params}, os.getcwd(), '{proxyURL}')\n"
                        )
                        cell["source"][i] = new_line

            for i, line in enumerate(cell["source"]):
                # 替换@param{type: 'xxx'}中的单引号为双引号
                if "@param" in line:
                    cell["source"][i] = line.replace("'", '"')

            for i, line in enumerate(cell["source"]):
                # Check if the line contains "wget" and add the import statement if necessary
                if "wget" in line and not import_added:
                    import_statements = [
                        "import sys",
                        "from IPython.utils import capture",
                        "sys.path.append('../../')",
                        "from func.env import setProxyCLI,proxyWget",
                        "cb=setProxyCLI()",
                        "proxy=cb['proxy']",
                        "region=cb['region']",
                        "proxyURL=cb['proxyURL']",
                        "",
                    ]
                    import_statements_with_newline = [
                        s + "\n" for s in import_statements
                    ]
                    # print(import_statements_with_newline)
                    cell["source"] = import_statements_with_newline + cell["source"]
                    # Set the flag to indicate that the import has been added
                    import_added = True

                # # 原始字符串
                # original = "!cp -r /openbayes/home/content/usr/local/lib/python3.8/dist-packages /usr/local/lib/python3.8/"
                # if original in line:
                #     # 替换python路径
                #     cell["source"][i] = re.sub(r'/usr/local/lib/python3.8/', '/usr/local/lib/python/', original)

    # 保存修改后的notebook
    newNotebook = modify_file_name(notebook)
    with open(newNotebook, "w") as f:
        json.dump(nb, f)

    return newNotebook


def check_environment(output):
    # 检查输出是否包含字符串"autodl"
    # print(output)
    global content_path
    for i in output:
        if "AutoDL" in i:
            print("当前运行环境为autodl")
            content_path = "/root/content"
            break
        # 检查输出是否包含字符串"openbayes"
        elif "OpenBayes" in i:
            print("当前运行环境为openbayes")
            content_path = "/openbayes/home/content"
            break
    return content_path


# 尝试执行第一个命令并存储输出
try:
    output1 = !cd /openbayes/home && /etc/welcome
    content_path = check_environment(output1)
except Exception as e:
    print("无法执行命令：", e)

# 尝试执行第二个命令并存储输出
try:
    output2 = !source /etc/autodl-motd
    content_path = check_environment(output2)
except Exception as e:
    print("无法执行命令：", e)

print("内容路径：", content_path)

# 在根目录下创建Content文件夹
content_folder = content_path
# 在当前运行的ipynb文件所在的目录下创建临时文件夹temp
if not os.path.exists(content_folder):
    os.makedirs(content_folder)

newNotebookPath = replace_root_path(
    notebook=fastDreamBoothPath, old_root="/content", new_root=content_folder
)
print(newNotebookPath)

# 对指定的变量名进行字符串和参数的替换

In [ ]:
import json
import os

def modify_file_name(file_path):
    # 获取文件名和文件扩展名
    file_name, file_ext = os.path.splitext(file_path)
    # 在文件名后加上 -modified
    modified_file_path = file_name + "-customData" + file_ext
    return modified_file_path


# 读取修改后的notebook
def update_ipynb_vars(inputFilePath, outputFilePath, var_map):
    with open(inputFilePath, "r") as f:
        notebook_data = json.load(f)

    # 用更新后的值替换变量赋值
    for cell in notebook_data["cells"]:
        if cell["cell_type"] == "code":
            for i in range(len(cell["source"])):
                for var, val in var_map.items():
                    if isinstance(val, str):
                        val = f'"{val}"'
                    # 有 #@ 说明这一行是提供给用户自行填充的参数，所以才进行参数替换
                    identifier = "#@"
                    thisLine = cell["source"][i]
                    if identifier in thisLine and (
                        thisLine.startswith(f"{var} =")
                        or thisLine.startswith(f"{var}=")
                    ):
                        annotation = thisLine.split(identifier)[1]
                        cell["source"][i] = f"{var}={val} {identifier+annotation}"

    # 保存修改后的notebook_data
    with open(outputFilePath, "w") as f:
        json.dump(notebook_data, f)

    # 此函数来验证文件是否已经修改成功
    # 读取修改后的文件
    with open(outputFilePath, "r") as f:
        modified_data = f.read()
    if notebook_data == modified_data:
        print("修改失败")
        return
    else:
        print("修改成功")
        return

    return outputFilePath


# 更新变量
replacements = {
    "Model_Version": "V2.1-768px",
    "Session_Name": "FuXingHao768",
    "Crop_size": 768,
    "Remove_existing_instance_images": False,
    "IMAGES_FOLDER_OPTIONAL": "/openbayes/home/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/FuXingHao768/instance_images",
    "Crop_images": False,
    "Resolution": 768,
}

# 更新变量
updatedDataNotebookPath = modify_file_name(newNotebookPath)
update_ipynb_vars(newNotebookPath, updatedDataNotebookPath, replacements)

# 检查最终版本的ipynb里面的赋值情况

In [ ]:
import json
import os


def print_with_color(string, keywords):
    # 创建一个空字符串用于存储结果
    result = ""

    # 遍历字符串中的每一个字眼
    for word in string.split(" "):
        if os.path.isabs(word.strip('"')):
            result += f"\033[37;42m{word}"
        elif word.isnumeric():
            result += f"\033[0;36m{word}"
        else:
            # 如果这个字眼是关键词，就将它的颜色更改为相应的颜色
            if word in keywords:
                result += f"{keywords[word]}{word}"
            # 否则，使用默认颜色
            else:
                result += f"\033[1;33m{word}"
        result += " \033[0m"  # 收尾都要加一个' \033[0m'
    return result


keywords = {
    "=": "\033[1;34m",  # &&：正常颜色
    "False": "\033[1;31m",  # git：红色
    "True": "\033[0;32m",  # user：绿色
    "config": "\033[1;33m",  # config：黄色
    "--global": "\033[1;34m",  # --global：蓝色
    '""': "\033[1;35m",  # http.proxy：紫色
    "user.email": "\033[1;36m",  # user.email：深蓝色
    "user.name": "\033[1;37m",  # user.name：深绿色
    "http.proxy": "\033[1;38m",  # http.proxy：深紫色
    "proxyURL": "\033[1;39m",  # proxyURL：深红色
    "cd": "\033[1;31m",  # cd：红色
    "dir_path": "\033[1;41m",  # dir_path：深黑色
}

# colored_string = print_with_color("git config --global user.email user@example.com", keywords)
# print(colored_string)


def detect_constant_assignments(inputFilePath: str):
    # 定义一个 set，用于记录已经出现过的字符串
    seen_strings = set()
    empty_strings = set()
    # 打开文件并读取数据
    with open(inputFilePath, "r") as f:
        ipynb_data = json.load(f)

    # 遍历所有的代码单元
    for cell in ipynb_data["cells"]:
        if cell["cell_type"] == "code":
            # 遍历当前代码单元中的所有行
            for line in cell["source"]:
                # 检测是否是赋值行，并获取变量名和值
                parts = line.split("=")
                if len(parts) == 2 and "@param" in line:
                    var_name = parts[0].strip()
                    value = parts[1].strip()
                    before, after = value.split("#@para")
                    # 如果 parts[0] 还没有出现过，就输出
                    if parts[0] not in seen_strings:
                        seen_strings.add(parts[0])
                        print(
                            f"\033[1;37m{var_name}"
                            + f'{ print_with_color(f" = {before}",keywords) }'
                            + "\033[1;30m"
                            + f"#@para{after}"
                        )


detect_constant_assignments(updatedDataNotebookPath)

# 可以用线上的这个工具来进行Escape Code Color的合成：https://ansi.gabebanks.net/
def refsColor():
    print("\033[1;33m \t\t\tprint颜色参考 \033[0m \n");
    print("\033[1;33m \t 字体颜色:\n");
    print("\033[0m none:\n");
    print("\033[0;30m back:\n");
    print("\033[1;30m dark_back:\n");
    print("\033[0;34m blue:\n");
    print("\033[1;34m light_blue:\n");
    print("\033[0;32m green:\n");
    print("\033[1;32m light_green:\n");
    print("\033[0;36m cyan:\n");
    print("\033[1;36m light_scan:\n");
    print("\033[0;31m red:\n");
    print("\033[1;31m light_read:\n");
    print("\033[0;35m purple:\n");
    print("\033[1;35m light_purple:\n");
    print("\033[0;33m brown:\n");
    print("\033[1;33m yellow:\n");
    print("\033[0;37m light_yellow:\n");
    print("\033[1;37m white:\n");
    print("\033[1;33m \t 背景颜色:\n");
    print("\033[0m none:\n");
    print("\033[0;40m back:\n");
    print("\033[0;44m blue:\n");
    print("\033[0;42m green:\n");
    print("\033[0;46m cyan:\n");
    print("\033[0;41m red:\n");
    print("\033[0;45m purple:\n");
    print("\033[0;43m brown:\n");
    print("\033[0;47m light_yellow:\033[0m\n");
    print("\033[1;33m \t 背景字体颜色:\n");
    print("\033[47;31m hello world\033[?25l");
    print("\033[42;50m hello world\033[?25l");

# refsColor()